In [1]:
# general
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import cv2
import pandas as pd
import tifffile as tiff
import random
from sklearn.metrics import jaccard_similarity_score

# work with polygons
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from shapely.wkt import loads as wkt_loads

# CNN
from keras.models import Model, load_model
import os
from baseline2 import generate_mask_for_image_and_class, get_unet

from keras import backend as K

Using Theano backend.
ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 0 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device gpu failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1



RuntimeError: ('initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1\n', 'You asked to force this device and it failed. No fallback to the cpu or other gpu device.')

In [ ]:
inDir = '/home/PycharmProjects/dstl-fcn/noize_kernel'
DF = pd.read_csv('../input/train_wkt_v4.csv')
GS = pd.read_csv('../input/grid_sizes.csv', names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
SB = pd.read_csv('../input/submission_empty.csv')


N_Cls = 10
ISZ = 160
smooth = 1e-12
trs = [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]

In [ ]:
def stretch_n(bands, lower_percent=0, higher_percent=100):
    out = np.zeros_like(bands, dtype=np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0  # np.min(band)
        b = 1  # np.max(band)
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)

def M(image_id):
    # __author__ = amaia
    # https://www.kaggle.com/aamaia/dstl-satellite-imagery-feature-detection/rgb-using-m-bands-example
    img = tiff.imread('../input/sixteen_band/{}_M.tif'.format(image_id))
    img = np.rollaxis(img, 0, 3)
    return img

def predict_id(id, model, trs):
    img = M(id)
    x = stretch_n(img)

    cnv = np.zeros((960, 960, 8)).astype(np.float32)
    prd = np.zeros((N_Cls, 960, 960)).astype(np.float32)
    cnv[:img.shape[0], :img.shape[1], :] = x

    for i in range(0, 6):
        line = []
        for j in range(0, 6):
            line.append(cnv[i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ])

        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model.predict(x, batch_size=4)
        for j in range(tmp.shape[0]):
            prd[:, i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ] = tmp[j]

    trs = [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]
    trs = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
    for i in range(N_Cls):
        prd[i] = prd[i] > trs[i]

    return prd[:, :img.shape[0], :img.shape[1]]

def calc_jac(pred, label):
    itersec = np.multiply(pred, label).sum()
    summ = (pred+label).sum()
    jac = itersec/(summ-itersec)
    return jac


def jaccard_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

In [ ]:
model = load_model('weights/unet_42quality.hdf5', 
                   custom_objects={'jaccard_coef': jaccard_coef,
                                   'jaccard_coef_int': jaccard_coef_int})

In [ ]:
s = 835

for IM_ID in os.listdir('M/train/'):
    msk = predict_id(IM_ID[:-4], model, trs)
    img = M(IM_ID[:-4])
    
    mask = generate_mask_for_image_and_class((img.shape[0], img.shape[1]), IM_ID[:-4], 3)[:s, :s]
    
    fig, axes = plt.subplots(ncols=3, figsize=(50, 50))
    ax = axes.ravel()
    
    ax[0].imshow(img[:, :, 5])
    ax[0].set_title('rgb') 
    ax[0].axis('off')
    
    ax[1].imshow(mask)
    ax[1].set_title('Road')    
    ax[1].axis('off')

    ax[2].imshow(msk[2], cmap=plt.get_cmap('gray'))
    ax[2].set_title('Misc. Manmade structures') 
    ax[2].axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
for IM_ID in os.listdir('M/test/'):
    msk = predict_id(IM_ID[:-4], model, trs)
    img = M(IM_ID[:-4])
    mask = generate_mask_for_image_and_class((img.shape[0], img.shape[1]), IM_ID[:-4], 1)[:s, :s]
    
    fig, axes = plt.subplots(ncols=2, figsize=(50, 50))
    ax = axes.ravel()
    
    ax[0].imshow(img[:, :, 5])
    ax[0].set_title('rgb') 
    ax[0].axis('off')

    ax[1].imshow(msk[0], cmap=plt.get_cmap('gray'))
    ax[1].set_title('Misc. Manmade structures') 
    ax[1].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:




plt.figure()
ax1 = plt.subplot(131)
ax1.set_title('image ID:6120_2_3')
ax1.imshow(img[:, :, 5])
ax2 = plt.subplot(132)
ax2.set_title('predict bldg pixels')
ax2.imshow(msk[0], cmap=plt.get_cmap('gray'))
# ax3 = plt.subplot(133)
# ax3.set_title('predict bldg polygones')
# ax3.imshow(mask_for_polygons(mask_to_polygons(msk[0], epsilon=1), img.shape[:2]), cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
# Buildings - large building, residential, non-residential, fuel storage facility, fortified building
# Misc. Manmade structures 
# Road 
# Track - poor/dirt/cart track, footpath/trail
# Trees - woodland, hedgerows, groups of trees, standalone trees
# Crops - contour ploughing/cropland, grain (wheat) crops, row (potatoes, turnips) crops
# Waterway 
# Standing water
# Vehicle Large - large vehicle (e.g. lorry, truck,bus), logistics vehicle
# Vehicle Small - small vehicle (car, van), motorbike

for IM_ID in os.listdir('M/train/'):
    
    msk = predict_id(IM_ID[:-4], model, trs)
    img = M(IM_ID[:-4], mode='train')
    
    fig, axes = plt.subplots(ncols=8, figsize=(100, 100))
    ax = axes.ravel()

    ax[0].imshow(img[:, :, 5])
    ax[0].set_title('msk') 
    ax[0].axis('off')

    ax[1].imshow(msk[0], cmap=plt.get_cmap('gray'))
    ax[1].set_title('Misc. Manmade structures') 
    ax[1].axis('off')

    ax[2].imshow(msk[1], cmap=plt.get_cmap('gray'))
    ax[2].set_title('Road')    
    ax[2].axis('off')

    ax[3].imshow(msk[2], cmap=plt.get_cmap('gray'))
    ax[3].set_title('Track')
    ax[3].axis('off')

    ax[4].imshow(msk[3], cmap=plt.get_cmap('gray'))
    ax[4].set_title('Crops') 
    ax[4].axis('off')

    ax[5].imshow(msk[4], cmap=plt.get_cmap('gray'))
    ax[5].set_title('Standing water')  
    ax[5].axis('off')

    ax[6].imshow(msk[5], cmap=plt.get_cmap('gray'))
    ax[6].set_title('Vehicle Large')
    ax[6].axis('off')

    ax[7].imshow(msk[6], cmap=plt.get_cmap('gray'))
    ax[7].set_title('Vehicle Small')
    ax[7].axis('off')
    plt.tight_layout()
    plt.show() 

In [ ]:
for IM_ID in os.listdir('M/test/'):
    

    msk = predict_id(IM_ID[:-4], model, trs, mode='test')
    img = M(IM_ID[:-4], mode='test')
    
    fig, axes = plt.subplots(ncols=8, figsize=(100, 100))
    ax = axes.ravel()

    # sigmoid 
    ax[0].imshow(img[:, :, 5])
    ax[0].set_title('msk') 
    ax[0].axis('off')

    ax[1].imshow(msk[0], cmap=plt.get_cmap('gray'))
    ax[1].set_title('Misc. Manmade structures') 
    ax[1].axis('off')

    ax[2].imshow(msk[1], cmap=plt.get_cmap('gray'))
    ax[2].set_title('Road')    
    ax[2].axis('off')

    ax[3].imshow(msk[2], cmap=plt.get_cmap('gray'))
    ax[3].set_title('Track')
    ax[3].axis('off')

    ax[4].imshow(msk[3], cmap=plt.get_cmap('gray'))
    ax[4].set_title('Crops') 
    ax[4].axis('off')

    ax[5].imshow(msk[4], cmap=plt.get_cmap('gray'))
    ax[5].set_title('Standing water')  
    ax[5].axis('off')

    ax[6].imshow(msk[5], cmap=plt.get_cmap('gray'))
    ax[6].set_title('Vehicle Large')
    ax[6].axis('off')

    ax[7].imshow(msk[6], cmap=plt.get_cmap('gray'))
    ax[7].set_title('Vehicle Small')
    ax[7].axis('off')
    plt.tight_layout()
    plt.show() 